In [1]:
!pip install transformers
!pip install pytorch-pretrained-bert
!pip install keybert
!pip install keybert[spacy]
!pip install ultralytics
!pip install gradio
!pip install git+https://github.com/openai/whisper.git
!sudo apt update
!sudo apt install ffmpeg
!pip install -q supervision==0.19.0rc3
!pip install -q inference-gpu[yolo-world]==0.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [2]:
from keybert import KeyBERT
import whisper
#from ultralytics import YOLOWorld
import time
import cv2
from transformers import pipeline
import supervision as sv
from tqdm import tqdm
from inference.models.yolo_world.yolo_world import YOLOWorld

kw_model = KeyBERT()
model = YOLOWorld(model_id="yolo_world/l")
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")
# Стоп-слова в виде списка


[05/26/24 12:25:54] WARNING  Your inference package version 0.11.0 is out of date! Please upgrade to ]8;id=869106;file:///usr/local/lib/python3.10/dist-packages/inference/core/__init__.py\__init__.py]8;;\:]8;id=612365;file:///usr/local/lib/python3.10/dist-packages/inference/core/__init__.py#35\35]8;;\
                             version 0.11.2 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [3]:
#Указываем список неактуальных для извлечения ключевых слов.
#Это могут быть любые слова, но были выбраны те, которые не несут в себе уникальности и нужны лишь для грамотного написания запроса.
#...И другие базовые стоп-слова, например союзы и частицы.
stop_words_list = ["this", "This", "and","And", "The", "the", "on", "On", "in", "In", "locate", "identify", "spot", "find", "discover", "pinpoint","Locate","Identify","Spot","Find","Discover", "Pinpoint", "image", "Image", "Picture", "picture"]

In [4]:
import gradio as gr
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
#Функция для обработки текстового запроса. Получение списка ключевых слов из него, с помощью KeyBERT.
def process_text_query(text):
    start_time = time.time()
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=stop_words_list)
    end_time = time.time()
    execution_time = (end_time - start_time) * 1000  # Время в миллисекундах
    filtered_keywords = [word for word, score in keywords if score > 0.05]

    if not filtered_keywords:
        raise ValueError("No valid keywords extracted from the text.")

    model.set_classes(filtered_keywords)
    print(f"Filtered Keywords: {filtered_keywords}")  # Отладочная информация
    return filtered_keywords

#Функция для обработки голосового запроса. Получение из него текстого запроса, с помощью Whisper.
#Передача этого текстового запроса в соответствующиу функцию для обработки
def process_voice_query(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    return transcriber({"sampling_rate": sr, "raw": y})["text"]
#Основная функция которая подвязана к нажатию кнопки "Обработать" в интерфейсе.
#Используется для обработки изображения, с помощью YOLO-WORLD и текстового запроса
def detect_objects_text(image, text_query):
    try:
        keywords = process_text_query(text_query)

        # Детектирование объектов
        results = model.infer(image, confidence=0.003)
        detections = sv.Detections.from_inference(results).with_nms(threshold=0.1)
        labels = [
            f"{keywords[class_id]} {confidence:0.3f}"
            for class_id, confidence
            in zip(detections.class_id, detections.confidence)
        ]

        BOUNDING_BOX_ANNOTATOR = sv.BoundingBoxAnnotator(thickness=2)
        LABEL_ANNOTATOR = sv.LabelAnnotator(text_thickness=2, text_scale=1, text_color=sv.Color.BLACK)

        annotated_image = image.copy()
        annotated_image = BOUNDING_BOX_ANNOTATOR.annotate(annotated_image, detections)
        annotated_image = LABEL_ANNOTATOR.annotate(annotated_image, detections, labels=labels)
        sv.plot_image(annotated_image, (10, 10))
        return annotated_image, keywords
    except Exception as e:
        print(f"Error: {e}")
        return image, [str(e)]

#Основная функция которая подвязана к нажатию кнопки "Обработать" в интерфейсе.
#Используется для обработки изображения, с помощью YOLO-WORLD и голосового запроса
def detect_objects_voice(image, audio_input):
    try:
        transcription = process_voice_query(audio_input)
        keywords = process_text_query(transcription)

        # Детектирование объектов
        results = model.infer(image, confidence=0.003)
        #использование NMS алгоритма
        detections = sv.Detections.from_inference(results).with_nms(threshold=0.1)
        labels = [
            f"{keywords[class_id]} {confidence:0.3f}"
            for class_id, confidence
            in zip(detections.class_id, detections.confidence)
        ]
        #Создание рамок bbox'a и названий обнаруженных классов(меток)
        BOUNDING_BOX_ANNOTATOR = sv.BoundingBoxAnnotator(thickness=2)
        LABEL_ANNOTATOR = sv.LabelAnnotator(text_thickness=2, text_scale=1, text_color=sv.Color.BLACK)

        annotated_image = image.copy()
        annotated_image = BOUNDING_BOX_ANNOTATOR.annotate(annotated_image, detections)
        annotated_image = LABEL_ANNOTATOR.annotate(annotated_image, detections, labels=labels)
        sv.plot_image(annotated_image, (10, 10))
        return annotated_image, transcription, keywords
    except Exception as e:
        print(f"Error: {e}")
        return image, f"Error: {str(e)}", [str(e)]

# Создание интерфейса с помощью Gradio.
# В данном интерфейсе подразумевается разделение вкладок "Голосового запроса" и "Текстового запроса". Хотя при желании можно объединить их.
# Главные функции для этих запросов разные, так как хотелось возвращать для голосового запроса еще и расшифрованный(переведенный) текстовый формат.
with gr.Blocks() as demo:
    gr.Markdown("## Объектное детектирование с помощью YOLO-WORLD")
    with gr.Tab("Текстовый запрос"):
        text_query = gr.Textbox(label="Введите текстовый запрос")
        image_input = gr.Image(type="numpy", label="Загрузите изображение")
        submit_btn = gr.Button("Обработать")
        output_image = gr.Image(type="numpy")
        output_text = gr.Textbox(label="Ключевые слова")
        submit_btn.click(detect_objects_text, inputs=[image_input, text_query], outputs=[output_image, output_text])

    with gr.Tab("Голосовой запрос"):
        audio_input = gr.Audio(type="numpy", label="Запишите голосовой запрос")
        image_input_voice = gr.Image(type="numpy", label="Загрузите изображение")
        submit_btn_voice = gr.Button("Обработать")
        output_voice_image = gr.Image(type="numpy")
        output_voice_text = gr.Textbox(label="Распознанный текст")
        output_voice_keywords = gr.Textbox(label="Ключевые слова")
        submit_btn_voice.click(detect_objects_voice, inputs=[image_input_voice, audio_input], outputs=[output_voice_image, output_voice_text, output_voice_keywords])

demo.launch(debug=True)

**Описание**

Реализованная система поддерживает хорошую работоспособность, как при запуске непосредственно из этого документа, так и при работе через ссылку(активную 72 часа), которая предоставляется Gradio для использования другими пользователями.



Приведенные ниже примеры работы с изображениями демонстрируют обработку классов неприсутстующих в COCO датасете. Тем самым демонстрируется неограниченность работы по сравнению с предыдущими реализациями.


![](https://drive.google.com/uc?export=view&id=1HW8TVsoBFAFCPrBZ6KODLgIePCbqFNjg)
![](https://drive.google.com/uc?export=view&id=1hAujIrNJx2idSpt3Qrf82RoF892L57W2)
![](https://drive.google.com/uc?export=view&id=1LQSkp5SAksh8wDXkkQv7RXIhP4Tw7xG7)
![](https://drive.google.com/uc?export=view&id=1HvBQYkwy7qmUvctbO8bj-NoqwtsBU-I8)
![](https://drive.google.com/uc?export=view&id=1x1UYZbTJ9jCJpiAkifgbtvgrXhtdgoj5)
![](https://drive.google.com/uc?export=view&id=1ByPHZk0zRQUOc1-BaxEXclrnkTDWvOl6)
![](https://drive.google.com/uc?export=view&id=1P4rMS9LjjxA_d_S5S-fkKVeZwdlavj7G)


Конечно, нетрудно заметить, что точность в сравнении с классическими детекторами не очень большая. Однако в формате zero-shot это приемлемые значения. Также, видно, что есть ряд недостатков:
1. Требуется донастройка YOLO-WORLD для лучшего обнаружения различных малоактуальных классов. Нужно это для решения проблем с ложным и отсутствующим срабатыванием.
2. Необходима оптимизация системы для достижения меньшего времени инференса и меньшей траты вычислительных ресурсов(видеопамять).

Однако при всем этом система уже показывает неплохие результаты и способна стать базой для дополнительных надстроек и более гибких решений.